<a href="https://colab.research.google.com/github/raz0208/ModernBERT/blob/main/ModernBERT_TokenEmbedding_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extract embedding form inpot text using ModernBERT Version 1

In [1]:
# import required libraries
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

### Load NLP and ModernBert models

In [ ]:
# Load ModernBERT tokenizer and model from Hugging Face
MODEL_NAME = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

### Extract emmbedings based on full text

In [3]:
# Function to get inpout text and return full text embedding
def get_text_embedding(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (use CLS token for sentence-level embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, hidden_size]

    return cls_embedding.squeeze().numpy()

In [ ]:
### --- ### Sample text for test ### --- ###

# 1- This is an application about Breast Cancer.
# 2- Treating high blood pressure, high blood lipids, diabetes.
# 3- Heart failure, heart attack, stroke, aneurysm, peripheral artery disease, sudden cardiac arrest. Deaths: 17.9 million / 32% (2015)
# 4- Heart failure and stroke are common causes of death.

### Exacute the app and get output

In [4]:
# Example usage (Sentence: This is an application about Breast Cancer.)
if __name__ == "__main__":
    user_text = input("Enter your text: ")

    # Get sentence embedding
    full_text_embedding = get_text_embedding(user_text)
    print("\nSentence Embedding vector shape:", full_text_embedding.shape)
    print("Sentence Embedding (first 10 values):", full_text_embedding[:10])

Enter your text: This is an application about Breast Cancer.

Sentence Embedding vector shape: (768,)
Sentence Embedding (first 10 values): [ 0.42236355 -0.8862073  -0.6536694  -0.2981413  -0.5874422  -0.720903
 -0.8588484  -0.89695704  0.5856571  -0.9214181 ]


## Use Neo4j to connect the graph database

In [5]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 17.1 MB/s eta 0:00:00


In [8]:
from neo4j import GraphDatabase

# Define Neo4j connection credentials
NEO4J_URI = "neo4j://143.225.233.156:7687"
NEO4J_USER = "rezaazari"
NEO4J_PASSWORD = "rAzari987"

# Initialize the driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Function to test connection
def test_connection():
    with driver.session() as session:
        greeting = session.run("RETURN 'Connected to Neo4j' AS message").single()["message"]
        print(greeting)

if __name__ == "__main__":
    test_connection()

Connected to Neo4j


In [9]:
# Function to run Cypher query
def run_query(cypher_query, parameters=None):
    with driver.session() as session:
        result = session.run(cypher_query, parameters or {})
        return [record.data() for record in result]

# Query of showing example nodes
query = "MATCH (n) RETURN n LIMIT 5"
results = run_query(query)
for r in results:
    print(r)

{'n': {'date': '1-12-1987', 'journal': 'The Journal of Cell Biology', 'hub': 0.0, 'auth': 2.6175247320960168e-12, 'subjects': 'Articles', 'pmc': 'PMC2114721', 'abstract': 'Meiosis I in males of the Dipteran Sciara coprophila results in the nonrandom distribution of maternally and paternally derived chromosome sets to the two division products. Based on an earlier study (Kubai, D.F. 1982. J. Cell Biol. 93:655-669), I suggested that the meiosis I spindle does not play a direct role in the nonrandom sorting of chromosomes but that, instead, haploid sets are already separated in prophase nuclei well before the onset of spindle formation. Here I report more direct evidence that this hypothesis is true; this evidence was gained from ultrastructural reconstruction analyses of the arrangement of chromosomes in germ line nuclei (prophase nuclei in spermatogonia and spermatocytes) of males heterozygous for an X- autosome chromosome translocation. Because of this translocation, the maternal and p

In [27]:
# Function to find similar nodes using cosine similarity
def find_similar_nodes(text_embedding, top_n):
     embedding_list = text_embedding.tolist()
     cypher_query = """
     MATCH (n)-[:HAS_EMBEDDING]->(e:ABSTRACT)
     WHERE e.embedding IS NOT NULL
     WITH n, e, gds.similarity.cosine($sent_embedding, e.embedding) AS similarity
     RETURN n, similarity
     ORDER BY similarity DESC
     LIMIT $limit
     """
     parameters = {"sent_embedding": embedding_list, "limit": top_n}
     results = run_query(cypher_query, parameters)
     return results

In [28]:
# Call function to run similarity query
similar_nodes = find_similar_nodes(full_text_embedding, top_n=5)

# show the result
print(f"\nTop {len(similar_nodes)} similar nodes:")
for node_data in similar_nodes:
         print(f"Node: {node_data['n']}, Similarity: {node_data['similarity']:.4f}")


Top 5 similar nodes:
Node: {'date': '01-2-2022', 'journal': 'Lymphatic Research and Biology', 'hub': 0.0, 'auth': 2.261210583376669e-70, 'subjects': 'Original Articles', 'pmc': 'PMC8892971', 'abstract': 'Background: Measuring lymphedema with high accuracy is important for several reasons. The aim of this study was to assess the reliability and validity of three routinely used methods to estimate limb volumes. Methods and Results: Inverse water displacement, girth measurements, and Perometer measurements were executed. Although standard techniques were used, extra precautions were taken to maximize accuracy within and between observers. Water displacement, and girth and Perometer measurements resulted in standard deviations of 0.7%–0.8%, 0.5%, and 0.4%–1.0%, respectively. Conclusion: Measuring limb volumes using routine methods is not easy. Even under optimal conditions, the measurements are quite dependent on the observer. For practical situations, an easy, objective, and reliable met